In [ ]:
# Module imports
import sys
import mysql.connector
from mysql.connector import errorcode

In [ ]:
# enter the configs
config = {
    "user": "whatabook_user",
    "password": "MySQL8IsGreat!",
    "host": "127.0.0.1",
    "database": "whatabook",
    "raise_on_warnings": True
}

In [ ]:
# Create menues with def statements.
def show_menu():
    print("\n Menu")
    print("    1. View Books\n    2. View Store Locations\n    4. Exit Program")
    try:
        choice = int(input('      <Example enterbook listing>: '))
        return choice
    except ValueError:
        print("\n  Invalid entry...\n")
        sys.exit(0)

        # Show the book
def show_books(_cursor):
    _cursor.execute("SELECT book_id, book_name, author, details from book")
    books = _cursor.fetchall()
    print("\n  -- DISPLAYING BOOK LISTING --")
        for book in books:
        print("  Book Name: {}\n  Author: {}\n  Details: {}\n".format(book[0], book[1]))
        # Show the location
def show_locations(_cursor):
    _cursor.execute("SELECT store_id, locale from store")
    locations = _cursor.fetchall()
    print("\n  -- Store Places --")
    for location in locations:
        print("  Locale: {}\n".format(location[1]))
# You want to validate  the users
def validate_user():
    """ validate the users ID """
    try:
        user_id = int(input('\n      Enter ID for customers <Example 1 for user_id 1>: '))

        if user_id < 0 or user_id > 3:
            print("\n  Invalid entry...\n")
            sys.exit(0)

        return user_id
    except ValueError:
        print("\n  Invalid entry...\n")
        sys.exit(0)

def show_account_menu():
    try:
        print("\n      -- Customer Menu --")
        print("        1. Wishlist\n        2. Add Book\n        3. Main Menu")
        account_option = int(input('        <Example enter: 1 for wishlist>: '))

        return account_option
    except ValueError:
        print("\n  Invalid entry...\n")

        sys.exit(0)

def show_wishlist(_cursor, _user_id):
    _cursor.execute("SELECT user.user_id, user.first_name, user.last_name, book.book_id, book.book_name, book.author " + 
                    "FROM wishlist " + 
                    "INNER JOIN user ON wishlist.user_id = user.user_id " + 
                    "WHERE user.user_id = {}".format(_user_id))
    # Print your wishlist
    for book in wishlist:
        print("        Book Name: {}\n        Author: {}\n".format(book[4], book[5]))

try:
    db = mysql.connector.connect(**config)
    print("\n  Welcome to the WhatABook Application! ")
    user_selection = show_menu()

In [ ]:
    db.close()
    # Close program